# TP2: Machine Learning

### Imports

In [1]:
import pandas as pd
from datetime import datetime
import scipy.spatial
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt

### Data loading

In [2]:
print "* * * * * Data loading * * * * *"
print "loading station csv"
stationDF = pd.read_csv('../CSVs/station.csv')
print "loading trip train csv"
trainingSet = pd.read_csv('../CSVs/trip_train.csv')
print "loading trip test csv"
testingSet = pd.read_csv('../CSVs/trip_test.csv')

# DF DEL TP1
# (0 = Monday, 1 = Tuesday...)
print "loading dfSF_Bay csv"
dfSF_Bay = pd.read_csv('../CSVs/dfSF_Bay.csv')

* * * * * Data loading * * * * *
loading station csv
loading trip train csv
loading trip test csv
loading dfSF_Bay csv


## Basic data analysis

In [3]:
print "stationDF.shape: ", stationDF.shape
print "trainingSet.shape: ", trainingSet.shape
print "testingSet.shape: ", testingSet.shape
print "dfSF_Bay.shape: ", dfSF_Bay.shape

stationDF.shape:  (70, 7)
trainingSet.shape:  (549961, 11)
testingSet.shape:  (119998, 10)
dfSF_Bay.shape:  (733, 33)


stationDF.shape:  (70, 7)<br>
trainingSet.shape:  (549961, 11)<br>
testingSet.shape:  (119998, 10)<br>
dfSF_Bay.shape:  (733, 33)<br>

### Distancias entre estaciones

In [4]:
print "* * * * * Working on station distances * * * * *"
# Create new temporary dataframe with distances
distancesDF = pd.DataFrame(columns=["start_station_id", "end_station_id", "distance"])

# Calculate distances between stations
for station, lat, lon in zip(stationDF.id, stationDF.lat, stationDF.long):
    for station2, lat2, lon2 in zip(stationDF.id, stationDF.lat, stationDF.long):
        distancesDF = distancesDF.append({
            "start_station_id": station,
            "end_station_id": station2,
            "distance": scipy.spatial.distance.cityblock([lat, lon], [lat2, lon2])
        }, ignore_index=True)

distancesDF['start_station_id'] = distancesDF.start_station_id.astype(int)
distancesDF['end_station_id'] = distancesDF.end_station_id.astype(int)

# Merge this new data to training and testing sets
trainingSet = pd.merge(trainingSet,distancesDF,on =['start_station_id','end_station_id'],how = 'inner')
testingSet = pd.merge(testingSet,distancesDF,on =['start_station_id','end_station_id'],how = 'inner')

# delete auxiliary distances df
del distancesDF

* * * * * Working on station distances * * * * *


### Process date & time data

In [5]:
print "* * * * * Converting necessary data to dateTime * * * * *"
# Convert necessary data to dateTime
dfSF_Bay['date'] = pd.to_datetime(dfSF_Bay.date)

trainingSet['start_date'] = pd.to_datetime(trainingSet.start_date)
trainingSet['end_date'] = pd.to_datetime(trainingSet.end_date)

testingSet['start_date'] = pd.to_datetime(testingSet.start_date)
testingSet['end_date'] = pd.to_datetime(testingSet.end_date)

# Create new features related to date & time based on the unique 'date' feature
# Work with training set
trainingSet['start_dayOfWeek'] = trainingSet.start_date.dt.dayofweek
trainingSet['start_week'] = trainingSet.start_date.dt.week
trainingSet['start_quarter'] = trainingSet.start_date.dt.quarter
trainingSet['start_time'] = trainingSet.start_date.dt.time
trainingSet['start_hour'] = trainingSet.start_date.dt.hour
trainingSet['start_minute'] = trainingSet.start_date.dt.minute
trainingSet['start_year'] = trainingSet.start_date.dt.year
trainingSet['start_month'] = trainingSet.start_date.dt.month
trainingSet['start_day'] = trainingSet.start_date.dt.day
trainingSet['start_date'] = trainingSet.start_date.dt.date

trainingSet['end_dayOfWeek'] = trainingSet.end_date.dt.dayofweek
trainingSet['end_week'] = trainingSet.end_date.dt.week
trainingSet['end_quarter'] = trainingSet.end_date.dt.quarter
trainingSet['end_time'] = trainingSet.end_date.dt.time
trainingSet['end_hour'] = trainingSet.end_date.dt.hour
trainingSet['end_minute'] = trainingSet.end_date.dt.minute
trainingSet['end_year'] = trainingSet.end_date.dt.year
trainingSet['end_month'] = trainingSet.end_date.dt.month
trainingSet['end_day'] = trainingSet.end_date.dt.day
trainingSet['end_date'] = trainingSet.end_date.dt.date

trainingSet['year'] = pd.to_datetime(trainingSet['start_date']).dt.year
trainingSet['month'] = pd.to_datetime(trainingSet['start_date']).dt.month
trainingSet['weekday'] = pd.to_datetime(trainingSet['start_date']).dt.weekday

# Work with testing set
testingSet['start_dayOfWeek'] = testingSet.start_date.dt.dayofweek
testingSet['start_week'] = testingSet.start_date.dt.week
testingSet['start_quarter'] = testingSet.start_date.dt.quarter
testingSet['start_time'] = testingSet.start_date.dt.time
testingSet['start_hour'] = testingSet.start_date.dt.hour
testingSet['start_minute'] = testingSet.start_date.dt.minute
testingSet['start_year'] = testingSet.start_date.dt.year
testingSet['start_month'] = testingSet.start_date.dt.month
testingSet['start_day'] = testingSet.start_date.dt.day
testingSet['start_date'] = testingSet.start_date.dt.date

testingSet['end_dayOfWeek'] = testingSet.end_date.dt.dayofweek
testingSet['end_week'] = testingSet.end_date.dt.week
testingSet['end_quarter'] =testingSet.end_date.dt.quarter
testingSet['end_time'] = testingSet.end_date.dt.time
testingSet['end_hour'] = testingSet.end_date.dt.hour
testingSet['end_minute'] = testingSet.end_date.dt.minute
testingSet['end_year'] = testingSet.end_date.dt.year
testingSet['end_month'] = testingSet.end_date.dt.month
testingSet['end_day'] = testingSet.end_date.dt.day
testingSet['end_date'] = testingSet.end_date.dt.date

testingSet['year'] = pd.to_datetime(testingSet['start_date']).dt.year
testingSet['month'] = pd.to_datetime(testingSet['start_date']).dt.month
testingSet['weekday'] = pd.to_datetime(testingSet['start_date']).dt.weekday

* * * * * Converting necessary data to dateTime * * * * *


In [6]:
print "trainingSet cols values", list(trainingSet.columns.values)

trainingSet cols values ['id', 'duration', 'start_date', 'start_station_name', 'start_station_id', 'end_date', 'end_station_name', 'end_station_id', 'bike_id', 'subscription_type', 'zip_code', 'distance', 'start_dayOfWeek', 'start_week', 'start_quarter', 'start_time', 'start_hour', 'start_minute', 'start_year', 'start_month', 'start_day', 'end_dayOfWeek', 'end_week', 'end_quarter', 'end_time', 'end_hour', 'end_minute', 'end_year', 'end_month', 'end_day', 'year', 'month', 'weekday']


In [7]:
print "testingSet cols values", list(testingSet.columns.values)

testingSet cols values ['id', 'start_date', 'start_station_name', 'start_station_id', 'end_date', 'end_station_name', 'end_station_id', 'bike_id', 'subscription_type', 'zip_code', 'distance', 'start_dayOfWeek', 'start_week', 'start_quarter', 'start_time', 'start_hour', 'start_minute', 'start_year', 'start_month', 'start_day', 'end_dayOfWeek', 'end_week', 'end_quarter', 'end_time', 'end_hour', 'end_minute', 'end_year', 'end_month', 'end_day', 'year', 'month', 'weekday']


## Feature Historico

In [8]:
print "* * * * * Working on historic feature * * * * *"
print "* * * Calculating historic feature * * *"
import math
listaStart = []
listaEnd = []
for i in list(trainingSet.start_station_id.values):
    if i not in listaStart:
        listaStart.append(i)
for i in list(trainingSet.end_station_id.values):
    if i not in listaEnd:
        listaEnd.append(i)
listaHistorico = []
for i in listaStart:
    for j in listaEnd:
        df = trainingSet[(trainingSet['start_station_id'] == i) & (trainingSet['end_station_id'] == j)]
        historico = df.duration.mean()
        if (not(math.isnan(historico))):
            listaHistorico.append([i,j,historico])
        else:
            listaHistorico.append([i,j,0])

listaHistorico

* * * * * Working on historic feature * * * * *
* * * Calculating historic feature * * *


[[50, 60, 1254.993877147936],
 [50, 46, 2679.90099009901],
 [50, 62, 2177.8607350096713],
 [50, 55, 502.29606625258799],
 [50, 39, 1938.610909090909],
 [50, 70, 903.78094462540719],
 [50, 61, 662.55546241967033],
 [50, 75, 955.34452296819791],
 [50, 67, 2432.1732026143791],
 [50, 72, 7406.0220588235297],
 [50, 51, 1104.3922651933701],
 [50, 76, 1278.5260663507108],
 [50, 50, 6343.4900849858359],
 [50, 66, 3062.5027027027027],
 [50, 54, 788.76531671858777],
 [50, 64, 842.403162055336],
 [50, 65, 1070.6219047619047],
 [50, 45, 846.55012224938878],
 [50, 28, 0],
 [50, 6, 0],
 [50, 77, 834.04506699147385],
 [50, 82, 1168.3443396226414],
 [50, 74, 4673.119565217391],
 [50, 2, 0],
 [50, 68, 968.3078101071975],
 [50, 71, 2404.4898477157362],
 [50, 56, 2053.1680000000001],
 [50, 49, 453.73065015479875],
 [50, 57, 925.07014028056108],
 [50, 63, 661.69466882067854],
 [50, 48, 1801.127619047619],
 [50, 69, 1076.7346760070052],
 [50, 11, 0],
 [50, 47, 1400.2985074626865],
 [50, 32, 0],
 [50, 31, 0

In [9]:
starStationId = []
endStationId = []
historical = []
for x in listaHistorico:
    starStationId.append(x[0])
    endStationId.append(x[1])
    historical.append(x[2])

data = {
    'start_station_id' : starStationId,
    'end_station_id' : endStationId,
    'historical' : historical,
}

dfData = pd.DataFrame(data,columns = ['start_station_id','end_station_id','historical'])
dfData

,start_station_id,end_station_id,historical
0,50,60,1254.993877
1,50,46,2679.900990
2,50,62,2177.860735
3,50,55,502.296066
4,50,39,1938.610909
5,50,70,903.780945
6,50,61,662.555462
7,50,75,955.344523
8,50,67,2432.173203
9,50,72,7406.022059


In [10]:
print "* * Merging historic feature * *"
# Merge this new data to training and testing dfs
# Training
trainingSet = pd.merge(trainingSet,dfData,on=['start_station_id', 'end_station_id'],how = 'inner') 

trainingSet['historical'] = trainingSet.historical.astype(int)

# Testing
testingSet = pd.merge(testingSet, dfData, on=['start_station_id', 'end_station_id'], how='inner')

testingSet['historical'] = testingSet.historical.astype(int)

# delete auxiliar dataframe
del dfData

* * Merging historic feature * *


In [11]:
print "trainingSet.shape: ", trainingSet.shape
print "testingSet.shape: ", testingSet.shape

trainingSet.shape:  (549961, 34)
testingSet.shape:  (119998, 33)


The difference in the shapes is due to the duration feature used in the training set, which was used to calculate the historical feature.

### Trabajamos con dfSF_Bay

In [12]:
# Convert necessary data to dateTime
dfSF_Bay['date'] = pd.to_datetime(dfSF_Bay.date)

trainingSet['start_date'] = pd.to_datetime(trainingSet.start_date)
trainingSet['end_date'] = pd.to_datetime(trainingSet.end_date)

testingSet['start_date'] = pd.to_datetime(testingSet.start_date)
testingSet['end_date'] = pd.to_datetime(testingSet.end_date)

In [13]:
print "***Merging dfSF_Bay data***"
# Merge trainingSet with new data

testingSet = pd.merge(testingSet,dfSF_Bay,left_on ='start_date',right_on='date',how = 'inner')
trainingSet = pd.merge(trainingSet,dfSF_Bay,left_on ='start_date',right_on='date',how = 'inner')

***Merging dfSF_Bay data***


In [14]:
print "Saving pre - Discretizacion csvs..."
trainingSet.to_csv('../CSVs/preDiscretizationTraining.csv', index=False)
testingSet.to_csv('../CSVs/preDiscretizationTesting.csv', index=False)

Saving pre - Discretizacion csvs...


# Discretizacion

In [15]:
(trainingSet.dtypes)

id                                     int64
duration                               int64
start_date                    datetime64[ns]
start_station_name                    object
start_station_id                       int64
end_date                      datetime64[ns]
end_station_name                      object
end_station_id                         int64
bike_id                                int64
subscription_type                     object
zip_code                              object
distance                             float64
start_dayOfWeek                        int64
start_week                             int64
start_quarter                          int64
start_time                            object
start_hour                             int64
start_minute                           int64
start_year                             int64
start_month                            int64
start_day                              int64
end_dayOfWeek                          int64
end_week  

In [16]:
list(testingSet.columns.values)

['id',
 'start_date',
 'start_station_name',
 'start_station_id',
 'end_date',
 'end_station_name',
 'end_station_id',
 'bike_id',
 'subscription_type',
 'zip_code',
 'distance',
 'start_dayOfWeek',
 'start_week',
 'start_quarter',
 'start_time',
 'start_hour',
 'start_minute',
 'start_year',
 'start_month',
 'start_day',
 'end_dayOfWeek',
 'end_week',
 'end_quarter',
 'end_time',
 'end_hour',
 'end_minute',
 'end_year',
 'end_month',
 'end_day',
 'year_x',
 'month_x',
 'weekday_x',
 'historical',
 'viajes',
 'max_temperature_c',
 'mean_temperature_c',
 'min_temperature_c',
 'max_dew_point_c',
 'mean_dew_point_c',
 'min_dew_point_c',
 'max_humidity',
 'mean_humidity',
 'min_humidity',
 'max_sea_level_pressure_cm',
 'mean_sea_level_pressure_cm',
 'min_sea_level_pressure_cm',
 'max_visibility_km',
 'mean_visibility_km',
 'min_visibility_km',
 'max_wind_Speed_kmh',
 'mean_wind_speed_kmh',
 'max_gust_speed_kmh',
 'precipitation_cm',
 'cloud_cover',
 'wind_dir_degrees',
 'Fog',
 'Fog-Rain',

In [17]:
print "* * * * * Discretizacion y Normalizacion * * * * *"
print "* * * * * Discretizacion * * * * *"

* * * * * Discretizacion y Normalizacion * * * * *
* * * * * Discretizacion * * * * *


In [18]:
def crearLista (listadoCompleto):
    listaReducida = []
    for i in listadoCompleto:
        if i not in listaReducida:
            listaReducida.append(i)
    listaReducida.sort()
    return listaReducida

In [19]:
def discretizar(columna,nombre, df):
    listaReducida = crearLista(columna)
    v = list(range(len(columna)))
    listaCompleta = list(columna)
    for i in listaReducida:
        for j in range(len(listaCompleta)):
            if(listaCompleta[j] == i):
                v[j] = 1
            else:
                v[j] = 0
        df[nombre+str(i)] = v

In [ ]:
print "Discretizando start_station_name..."
discretizar(trainingSet.start_station_name,'start ', trainingSet)
discretizar(testingSet.start_station_name,'start ', testingSet)

print "Discretizando end_station_name..."
discretizar(trainingSet.end_station_name,'end ', trainingSet)
discretizar(testingSet.end_station_name,'end ', testingSet)

print "Discretizando start_dayOfWeek..."
discretizar(trainingSet.start_dayOfWeek,'start_dayOfWeek_id', trainingSet)
discretizar(testingSet.start_dayOfWeek,'start_dayOfWeek_id', testingSet)

print "Discretizando end_dayOfWeek..."
discretizar(trainingSet.end_dayOfWeek,'end_dayOfWeek_id', trainingSet)
discretizar(testingSet.end_dayOfWeek,'end_dayOfWeek_id', testingSet)

print "Discretizando subscription_type_..."
discretizar(trainingSet.subscription_type,'subscription_type_', trainingSet)
discretizar(testingSet.subscription_type,'subscription_type_', testingSet)

print "Discretizando start_year..."
discretizar(trainingSet.start_year,'start_year_', trainingSet)
discretizar(testingSet.start_year,'start_year_', testingSet)

print "Discretizando end_year_..."
discretizar(trainingSet.end_year,'end_year_', trainingSet)
discretizar(testingSet.end_year,'end_year_', testingSet)

print "Discretizando start_month..."
discretizar(trainingSet.start_month,'start_month_', trainingSet)
discretizar(testingSet.start_month,'start_month_', testingSet)

print "Discretizando end_month..."
discretizar(trainingSet.end_month,'end_month_', trainingSet)
discretizar(testingSet.end_month,'end_month_', testingSet)

print "Discretizando start_day..."
discretizar(trainingSet.start_day,'start_day_', trainingSet)
discretizar(testingSet.start_day,'start_day_', testingSet)

print "Discretizando end_day..."
discretizar(trainingSet.end_day,'end_day_', trainingSet)
discretizar(testingSet.end_day,'end_day_', testingSet)

print "Discretizando start_quarter..."
discretizar(trainingSet.start_quarter,'start_quarter_', trainingSet)
discretizar(testingSet.start_quarter,'start_quarter_', testingSet)

print "Discretizando end_quarter..."
discretizar(trainingSet.end_quarter,'end_quarter_', trainingSet)
discretizar(testingSet.end_quarter,'end_quarter_', testingSet)

print "Discretizando start_hour..."
discretizar(trainingSet.start_hour,'start_hour_', trainingSet)
discretizar(testingSet.start_hour,'start_hour_', testingSet)

print "Discretizando end_hour..."
discretizar(trainingSet.end_hour,'end_hour', trainingSet)
discretizar(testingSet.end_hour,'end_hour', testingSet)

Discretizando start_station_name...
Discretizando end_station_name...
Discretizando start_dayOfWeek...
Discretizando end_dayOfWeek...
Discretizando subscription_type_...
Discretizando start_year...
Discretizando end_year_...
Discretizando start_month...
Discretizando end_month...
Discretizando start_day...
Discretizando end_day...
Discretizando start_quarter...
Discretizando end_quarter...
Discretizando start_hour...
Discretizando end_hour...


In [ ]:
print "Dropping trash columns..."
trainingSet = trainingSet.drop(labels = ['start_date', 
                                         'end_station_name',
                                         'start_station_name',
                                         'end_date',
                                         'subscription_type',
                                         'zip_code',
                                         'start_time',
                                         'end_time',
                                         'start_dayOfWeek',
                                         'end_dayOfWeek',
                                         'start_year',
                                         'end_year',
                                         'start_month',
                                         'end_month',
                                         'start_day',
                                         'end_day',
                                         'start_quarter',
                                         'end_quarter',
                                         'start_hour',
                                         'end_hour'
                                        ],axis = 1)

testingSet = testingSet.drop(labels = ['start_date', 
                                         'end_station_name',
                                         'start_station_name',
                                         'end_date',
                                         'subscription_type',
                                         'zip_code',
                                         'start_time',
                                         'end_time',
                                         'start_dayOfWeek',
                                         'end_dayOfWeek',
                                         'start_year',
                                         'end_year',
                                         'start_month',
                                         'end_month',
                                         'start_day',
                                         'end_day',
                                         'start_quarter',
                                         'end_quarter',
                                         'start_hour',
                                         'end_hour'
                                        ],axis = 1)

Dropping trash columns...


In [ ]:
print "trainingSet.shape: ", trainingSet.shape
print "testingSet.shape: ", testingSet.shape

In [ ]:
# THIS CELL SHOULD NOT BE COPIED. THIS CELL IS ALLOWED TO BE PRESENT ONLY ONCE IN ALL THE NOTEBOOK
print "Saving temp csvs..."
trainingSet.to_csv('../CSVs/tempTraining.csv', index=False)
testingSet.to_csv('../CSVs/tempTesting.csv', index=False)

# Binarizacion

In [ ]:
def binarizar(columna,name,df):
    lista = []
    for i in list(columna):
        numero = int(i)
        lista.append(int(bin(numero)[2:]))
    df[name] = lista

In [ ]:
print "Binarizando start_station_id..."
binarizar(trainingSet.start_station_id,'start_station_id', trainingSet)
binarizar(testingSet.start_station_id,'start_station_id', testingSet)

print "Binarizando end_station_id..."
binarizar(trainingSet.end_station_id,'end_station_id', trainingSet)
binarizar(testingSet.end_station_id,'end_station_id', testingSet)

# Data filtering

In [ ]:
trainingSet.drop(['Unnamed: 0'],1,inplace=True)
testingSet.drop(['Unnamed: 0'],1,inplace=True)

In [ ]:
# Delete repeated data
trainingSet.drop(['date','year_y','month_y','weekday_y'],1,inplace=True)
trainingSet = trainingSet.rename(columns={'year_x':'year','month_x':'month','weekday_x': 'weekday'})

testingSet.drop(['date','year_y','month_y','weekday_y'],1,inplace=True)
testingSet = testingSet.rename(columns={'year_x':'year','month_x':'month','weekday_x': 'weekday'})

In [ ]:
# Delete:
#     id: el id que identifica univocamente cada uno de los viajes 
#         no proporciona informacion con la que el algoritmo pueda aprender
#     start_station_id y end_station_id: las estaciones ya estan discretizadas por nombre
trainingSet.drop(['id', 'start_station_id','end_station_id'],1,inplace=True)

testingSet.drop(['id', 'start_station_id','end_station_id'],1,inplace=True)

In [ ]:
# Delete: 
#     bike_id: la duracion del viaje es independiente de la bicicleta, 
#         ya que son todas iguales ("") y se entregan sin juicio alguno ("")
trainingSet.drop(['bike_id'],1,inplace=True)

testingSet.drop(['bike_id'],1,inplace=True)

In [ ]:
# Delete: 
#     los dias como numero no aportan nada. E.g. 1 puede ser cualquier dia de la semana. 
trainingSet.drop(['start_day_1','start_day_2', 'start_day_3','start_day_4', 'start_day_5','start_day_6','start_day_7', 
                  'start_day_8','start_day_9', 'start_day_10','start_day_11', 'start_day_12','start_day_13', 
                  'start_day_14','start_day_15','start_day_16', 'start_day_17','start_day_18','start_day_19',
                  'start_day_20','start_day_21','start_day_22', 'start_day_23','start_day_24','start_day_25', 
                  'start_day_26','start_day_27','start_day_28','start_day_29','start_day_30',
                  'start_day_31'],1,inplace=True)
trainingSet.drop(['end_day_1','end_day_2','end_day_3','end_day_4','end_day_5','end_day_6','end_day_7','end_day_8',
                  'end_day_9','end_day_10', 'end_day_11','end_day_12', 'end_day_13','end_day_14','end_day_15',
                  'end_day_16','end_day_17','end_day_18','end_day_19','end_day_20','end_day_21','end_day_22',
                  'end_day_23','end_day_24','end_day_25','end_day_26','end_day_27','end_day_28','end_day_29',
                  'end_day_30','end_day_31',],1,inplace=True)

testingSet.drop(['start_day_1','start_day_2', 'start_day_3','start_day_4', 'start_day_5','start_day_6','start_day_7', 
                  'start_day_8','start_day_9', 'start_day_10','start_day_11', 'start_day_12','start_day_13', 
                  'start_day_14','start_day_15','start_day_16', 'start_day_17','start_day_18','start_day_19',
                  'start_day_20','start_day_21','start_day_22', 'start_day_23','start_day_24','start_day_25', 
                  'start_day_26','start_day_27','start_day_28','start_day_29','start_day_30',
                  'start_day_31'],1,inplace=True)
testingSet.drop(['end_day_1','end_day_2','end_day_3','end_day_4','end_day_5','end_day_6','end_day_7','end_day_8',
                  'end_day_9','end_day_10', 'end_day_11','end_day_12', 'end_day_13','end_day_14','end_day_15',
                  'end_day_16','end_day_17','end_day_18','end_day_19','end_day_20','end_day_21','end_day_22',
                  'end_day_23','end_day_24','end_day_25','end_day_26','end_day_27','end_day_28','end_day_29',
                  'end_day_30','end_day_31',],1,inplace=True)

In [ ]:
# Delete:
#     La duracion del viaje no puede depender de algo del final del mismo. 
#     De la misma manera, razonando analogamente, podemos concluir que contrario a esto, 
#     si influye el instante inicial del mismo
#     Retiro lo dicho para la estacion final, quedando valido el razonamiento unicamente 
#     para cuestiones temporales. Aun asi esto esta abierto a discusion.
trainingSet.drop(['end_week', 'end_minute','end_dayOfWeek_id0','end_dayOfWeek_id1','end_dayOfWeek_id2',
                  'end_dayOfWeek_id3','end_dayOfWeek_id4','end_dayOfWeek_id5','end_dayOfWeek_id6', 'end_year_2013',
                  'end_year_2014','end_year_2015','end_month_1','end_month_2','end_month_3','end_month_4',
                  'end_month_5','end_month_6','end_month_7','end_month_8','end_month_9','end_month_10',
                  'end_month_11','end_month_12','end_quarter_1','end_quarter_2','end_quarter_3','end_quarter_4',  'end_hour0',
                  'end_hour1', 'end_hour2','end_hour3','end_hour4','end_hour5','end_hour6','end_hour7','end_hour8','end_hour9',
                  'end_hour10','end_hour11','end_hour12','end_hour13','end_hour14','end_hour15','end_hour16',
                  'end_hour17','end_hour18','end_hour19','end_hour20','end_hour21','end_hour22','end_hour23' ],
                 1,inplace=True)

testingSet.drop(['end_week', 'end_minute','end_dayOfWeek_id0','end_dayOfWeek_id1','end_dayOfWeek_id2',
                  'end_dayOfWeek_id3','end_dayOfWeek_id4','end_dayOfWeek_id5','end_dayOfWeek_id6', 'end_year_2013',
                  'end_year_2014','end_year_2015','end_month_1','end_month_2','end_month_3','end_month_4',
                  'end_month_5','end_month_6','end_month_7','end_month_8','end_month_9','end_month_10',
                  'end_month_11','end_month_12','end_quarter_1','end_quarter_2','end_quarter_3','end_quarter_4',  'end_hour0',
                  'end_hour1', 'end_hour2','end_hour3','end_hour4','end_hour5','end_hour6','end_hour7','end_hour8','end_hour9',
                  'end_hour10','end_hour11','end_hour12','end_hour13','end_hour14','end_hour15','end_hour16',
                  'end_hour17','end_hour18','end_hour19','end_hour20','end_hour21','end_hour22','end_hour23' ],
                 1,inplace=True)

In [ ]:
# !!!!!!!! U L T I M A   C E L D A  !!!!!!!!
print "Saving to new csvs..."
print "Saving trainingSet to ../CSVs/finalTraining.csv..."
trainingSet.to_csv('../CSVs/finalTraining.csv')
print "Saving testingSet to ../CSVs/finalTesting.csv..."
testingSet.to_csv('../CSVs/finalTesting.csv')